No Aug Experiments - Keep batch size the same

In [1]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, augmentation = 0)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['noAugF'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.624. Accuracy is 8.60%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 3.053. Accuracy is 58.00%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.600. Accuracy is 81.41%
INFO:mlp.optimisers:Epoch 1: Took 8 seconds. Training speed 380 pps. Validation speed 1896 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.518. Accuracy is 83.50%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.497. Accuracy is 85.12%
INFO:mlp.optimisers:Epoch 2: Took 8 seconds. Training speed 460 pps. Validation speed 1819 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.324. Accuracy is 91.00%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.458. Accuracy is 85.38%
INFO:mlp.optimisers:Epoch 3: Took 8 seconds. Training speed

In [6]:
import shelve
#Go through values array and put into new list.
#Open file to save to
shelve_r = shelve.open("augExperiments")

print shelve_r['rotAugF0-8']






([(2.5675238888641019, 0.083499999999999949), (3.2944894821772159, 0.48649999999999999), (0.82832050674946944, 0.72899999999999976), (0.59143224278500872, 0.81800000000000028), (0.51179130588525079, 0.83550000000000013), (0.37741398617988331, 0.88750000000000062), (0.33053541301047562, 0.8965000000000003), (0.28140210893374124, 0.91150000000000009), (0.22197073930798447, 0.9340000000000005), (0.16961350593641591, 0.96000000000000041), (0.15102098322278623, 0.96500000000000075), (0.11934304052474623, 0.97050000000000025), (0.10134453418312969, 0.98050000000000026), (0.086864214560214495, 0.98300000000000043), (0.071798417991226826, 0.99050000000000027), (0.058238673193779407, 0.99300000000000044), (0.055229166570296438, 0.99350000000000005), (0.046314642352540972, 0.99500000000000011), (0.040450229474914175, 0.99650000000000005), (0.034288786107912667, 0.99750000000000005), (0.030358601152831378, 0.99750000000000005), (0.027791666206491025, 0.998), (0.026170635255226674, 0.9975000000000

Guassian augmentation

In [2]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, augmentation = 1)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['gauAugF'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.624. Accuracy is 8.60%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 3.053. Accuracy is 58.00%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.600. Accuracy is 81.41%
INFO:mlp.optimisers:Epoch 1: Took 8 seconds. Training speed 468 pps. Validation speed 1820 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.518. Accuracy is 83.50%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.497. Accuracy is 85.12%
INFO:mlp.optimisers:Epoch 2: Took 8 seconds. Training speed 370 pps. Validation speed 1843 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.324. Accuracy is 91.00%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.458. Accuracy is 85.38%
INFO:mlp.optimisers:Epoch 3: Took 8 seconds. Training speed

Rotation Augmentation

In [3]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, augmentation = 2)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['rotAugF'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.598. Accuracy is 8.90%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 3.215. Accuracy is 51.90%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.795. Accuracy is 74.86%
INFO:mlp.optimisers:Epoch 1: Took 9 seconds. Training speed 335 pps. Validation speed 1807 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.817. Accuracy is 73.25%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.650. Accuracy is 79.26%
INFO:mlp.optimisers:Epoch 2: Took 9 seconds. Training speed 315 pps. Validation speed 1748 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.617. Accuracy is 81.00%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.505. Accuracy is 84.57%
INFO:mlp.optimisers:Epoch 3: Took 8 seconds. Training speed

Dropout Augmentation

In [4]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, augmentation = 3)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['dpAugF'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.624. Accuracy is 8.60%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 2.851. Accuracy is 61.50%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.594. Accuracy is 81.14%
INFO:mlp.optimisers:Epoch 1: Took 8 seconds. Training speed 389 pps. Validation speed 1870 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.502. Accuracy is 84.10%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.460. Accuracy is 86.49%
INFO:mlp.optimisers:Epoch 2: Took 8 seconds. Training speed 373 pps. Validation speed 1825 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.334. Accuracy is 89.50%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.666. Accuracy is 80.03%
INFO:mlp.optimisers:Epoch 3: Took 8 seconds. Training speed

Image Shift Augmentation

In [5]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, augmentation = 4)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['siAugF'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.624. Accuracy is 7.50%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 3.589. Accuracy is 43.60%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.895. Accuracy is 71.53%
INFO:mlp.optimisers:Epoch 1: Took 9 seconds. Training speed 385 pps. Validation speed 1525 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.988. Accuracy is 69.05%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.573. Accuracy is 83.65%
INFO:mlp.optimisers:Epoch 2: Took 10 seconds. Training speed 325 pps. Validation speed 1525 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.770. Accuracy is 75.90%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.669. Accuracy is 79.51%
INFO:mlp.optimisers:Epoch 3: Took 9 seconds. Training spee

Random Augmentation

In [6]:
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateExponential, LearningRateFixed, LearningRateList, LearningRateNewBob

import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

#Set no aug
train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, augmentation = 5)
valid_dp = MNISTDataProvider(dset='valid', batch_size=10000, max_num_batches=-10, randomize=False)
test_dp = MNISTDataProvider(dset='eval', batch_size=10000, max_num_batches=-10, randomize=False)

rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 800
max_epochs = 30
cost = CECost()
learning_rate = 0.5;
learningList = []
decrement = (learning_rate/max_epochs)

#Regulariser weights
l1_weight = 0.00
l2_weight = 0.000
dp_scheduler = None

#Build list once so we don't have to rebuild every time.
for i in xrange(0,max_epochs):
    #In this order so start learning rate is added
    learningList.append(learning_rate)
    learning_rate -= decrement



#Open file to save to
shelve_r = shelve.open("augExperiments")

stats = []
rate = 2

#For each number of layers, new model add layers.
for layer in xrange(0,2):
    #Set here in case we alter it in a layer experiment
    learning_rate = 0.5


    train_dp.reset()
    valid_dp.reset()
    test_dp.reset()

    logger.info("Starting ")

    #define the model
    model = MLP(cost=cost)

    if layer == 0:
        odim = 800
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
    if layer == 1:
        odim = 300
        model.add_layer(Sigmoid(idim=784, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        model.add_layer(Sigmoid(idim=odim, odim=odim, irange=0.2, rng=rng))
        
    #Add output layer
    model.add_layer(Softmax(idim=odim, odim=10, rng=rng))

    #Set rate scheduler here
    if rate == 1:
        lr_scheduler = LearningRateExponential(start_rate=learning_rate, max_epochs=max_epochs, training_size=100)
    elif rate == 2:
        lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    elif rate == 3:
        # define the optimiser, here stochasitc gradient descent
        # with fixed learning rate and max_epochs
        lr_scheduler = LearningRateNewBob(start_rate=learning_rate, max_epochs=max_epochs,\
                                          min_derror_stop=.05, scale_by=0.05, zero_rate=learning_rate, patience = 10)

    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler, 
                             dp_scheduler=dp_scheduler,
                             l1_weight=l1_weight, 
                             l2_weight=l2_weight)
    
    logger.info('Training started...')
    tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

    logger.info('Testing the model on test set:')
    tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
    logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

    #Append stats for all test
    stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

    #Should save rate to specific dictionairy in pickle
    shelve_r['ranAugF'+str(layer)] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

logger.info('Saving Data')
shelve_r.close()   

INFO:root:Initialising data providers...
INFO:root:Starting 
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.621. Accuracy is 8.20%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 2.554. Accuracy is 9.84%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 3.020. Accuracy is 53.10%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 0.629. Accuracy is 79.91%
INFO:mlp.optimisers:Epoch 1: Took 9 seconds. Training speed 345 pps. Validation speed 1603 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 0.759. Accuracy is 76.70%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.566. Accuracy is 82.65%
INFO:mlp.optimisers:Epoch 2: Took 9 seconds. Training speed 326 pps. Validation speed 1590 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 0.627. Accuracy is 81.25%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.511. Accuracy is 84.96%
INFO:mlp.optimisers:Epoch 3: Took 9 seconds. Training speed